In [1]:
import pandas as pd
import numpy as np

In [37]:
chronos_base_path = "results/chronos/chronos_bolt_base_results.csv"
chronos_small_path = "results/chronos/chronos_bolt_small_results.csv"
chronos_mini_path = "results/chronos/chronos_bolt_mini_results.csv"
chronos_tiny_path = "results/chronos/chronos_bolt_tiny_results.csv"

timesfm1_path = "results/timesfm/timesfm1_results.csv"
timesfm2_path = "results/timesfm/timesfm2_results.csv"

In [38]:
def path_to_df(path):
    df = pd.read_csv(path)
    df = df[["dataset", "domain", "num_variates", "model",
        "eval_metrics/MASE[0.5]", "eval_metrics/mean_weighted_sum_quantile_loss"]]
    df = df.rename(columns={"eval_metrics/MASE[0.5]": "MASE"})
    df = df.rename(columns={"eval_metrics/mean_weighted_sum_quantile_loss": "Qloss"})
    return df

In [39]:
chronos_base_df = path_to_df(chronos_base_path)
chronos_base_df.head()

,dataset,domain,num_variates,model,MASE,Qloss
0,bizitobs_application/10S/short,Web/CloudOps,2,chronos_bolt_base,5.533290,0.053958
1,bizitobs_application/10S/medium,Web/CloudOps,2,chronos_bolt_base,9.720338,0.103502
2,bizitobs_application/10S/long,Web/CloudOps,2,chronos_bolt_base,10.483795,0.108976
3,bizitobs_l2c/5T/short,Web/CloudOps,7,chronos_bolt_base,0.278185,0.074353
4,bizitobs_l2c/5T/medium,Web/CloudOps,7,chronos_bolt_base,0.877710,0.445441


In [40]:
chronos_small_df = path_to_df(chronos_small_path)
chronos_mini_df = path_to_df(chronos_mini_path)
chronos_tiny_df = path_to_df(chronos_tiny_path)

timesfm1_df = path_to_df(timesfm1_path)
timesfm2_df = path_to_df(timesfm2_path)

In [41]:
# dataset with Mase
all_df = pd.concat([chronos_base_df, chronos_small_df, chronos_mini_df, chronos_tiny_df,
    timesfm1_df, timesfm2_df], ignore_index=True)

all_df["MASE"] = pd.to_numeric(all_df["MASE"], errors="coerce")
all_df["rank_MASE"] = all_df.groupby(["dataset"])["MASE"].rank(method="min")

all_df["Qloss"] = pd.to_numeric(all_df["Qloss"], errors="coerce")
all_df["rank_Qloss"] = all_df.groupby(["dataset"])["Qloss"].rank(method="min")

all_df["Rank"] = all_df["rank_MASE"] + all_df["rank_Qloss"] / 2

all_df = all_df[all_df["dataset"] != "dataset"]

all_df

,dataset,domain,num_variates,model,MASE,Qloss,rank_MASE,rank_Qloss,Rank
0,bizitobs_application/10S/short,Web/CloudOps,2,chronos_bolt_base,5.533290,0.053958,3.0,3.0,4.5
1,bizitobs_application/10S/medium,Web/CloudOps,2,chronos_bolt_base,9.720338,0.103502,3.0,3.0,4.5
2,bizitobs_application/10S/long,Web/CloudOps,2,chronos_bolt_base,10.483795,0.108976,3.0,3.0,4.5
3,bizitobs_l2c/5T/short,Web/CloudOps,7,chronos_bolt_base,0.278185,0.074353,3.0,3.0,4.5
4,bizitobs_l2c/5T/medium,Web/CloudOps,7,chronos_bolt_base,0.877710,0.445441,2.0,2.0,3.0
...,...,...,...,...,...,...,...,...,...
205,jena_weather/H/long,Nature,21,timesfm2,1.056757,0.068474,2.0,5.0,4.5
206,solar/10T/long,Energy,1,timesfm2,1.148564,0.498298,2.0,4.0,4.0
207,solar/H/long,Energy,1,timesfm2,1.266107,0.493456,5.0,5.0,7.5
208,bizitobs_application/10S/long,Web/CloudOps,2,timesfm2,4.085063,0.057008,1.0,1.0,1.5


In [42]:
all_df[all_df["dataset"] == "bizitobs_application/10S/short"]

,dataset,domain,num_variates,model,MASE,Qloss,rank_MASE,rank_Qloss,Rank
0,bizitobs_application/10S/short,Web/CloudOps,2,chronos_bolt_base,5.533290,0.053958,3.0,3.0,4.5
35,bizitobs_application/10S/short,Web/CloudOps,2,chronos_bolt_small,5.406632,0.034530,2.0,2.0,3.0
102,bizitobs_application/10S/short,Web/CloudOps,2,chronos_bolt_mini,7.201342,0.072335,4.0,5.0,6.5
113,bizitobs_application/10S/short,Web/CloudOps,2,chronos_bolt_tiny,7.547395,0.069873,5.0,4.0,7.0
153,bizitobs_application/10S/short,Web/CloudOps,2,timesfm1,1391.842484,28.269510,6.0,6.0,9.0
188,bizitobs_application/10S/short,Web/CloudOps,2,timesfm2,1.560993,0.014470,1.0,1.0,1.5


In [43]:
dataset_names = all_df["dataset"].unique()

In [44]:
dataset_names

array(['bizitobs_application/10S/short',
       'bizitobs_application/10S/medium', 'bizitobs_application/10S/long',
       'bizitobs_l2c/5T/short', 'bizitobs_l2c/5T/medium',
       'bizitobs_l2c/5T/long', 'jena_weather/10T/short',
       'jena_weather/10T/medium', 'jena_weather/10T/long',
       'bizitobs_service/10S/short', 'bizitobs_service/10S/medium',
       'bizitobs_service/10S/long', 'bizitobs_l2c/H/short',
       'bizitobs_l2c/H/medium', 'bizitobs_l2c/H/long', 'solar/D/short',
       'bitbrains_fast_storage/5T/short',
       'bitbrains_fast_storage/5T/medium',
       'bitbrains_fast_storage/5T/long', 'bitbrains_rnd/H/short',
       'bitbrains_fast_storage/H/short', 'bitbrains_rnd/5T/short',
       'bitbrains_rnd/5T/medium', 'bitbrains_rnd/5T/long',
       'jena_weather/H/short', 'jena_weather/H/medium',
       'jena_weather/H/long', 'solar/W/short', 'jena_weather/D/short',
       'solar/H/short', 'solar/H/medium', 'solar/H/long',
       'solar/10T/short', 'solar/10T/medium', 's

In [45]:
for name in dataset_names:
    #print(name)
    #print(all_df[all_df["dataset"] == name].to_latex(index=False, float_format="%.3f"))
    print(all_df[all_df["dataset"] == name][["model", "MASE", "rank_MASE", "Qloss", "rank_Qloss"]])
    print("\n")
    break

                  model         MASE  rank_MASE      Qloss  rank_Qloss
0     chronos_bolt_base     5.533290        3.0   0.053958         3.0
35   chronos_bolt_small     5.406632        2.0   0.034530         2.0
102   chronos_bolt_mini     7.201342        4.0   0.072335         5.0
113   chronos_bolt_tiny     7.547395        5.0   0.069873         4.0
153            timesfm1  1391.842484        6.0  28.269510         6.0
188            timesfm2     1.560993        1.0   0.014470         1.0




In [46]:
# new dataframe columns: "Chronos Bolt Base" "Chronos Bolt Small" "Chronos Bolt Mini" "Chronos Bolt Tiny" "TimesFM1" "TimesFM2"
# row: for each dataset name: 4 column (MAse, Qloss, rank_MASE, rank_Qloss, mean(rank_MASE, rank_Qloss))

def create_dataset_df(df, dataset_name):
    dataset_df = df[df["dataset"] == dataset_name]
    dataset_df = dataset_df.drop(columns=["dataset", "domain", "num_variates", "rank_MASE", "rank_Qloss"])
    dataset_df = dataset_df.rename(columns={"model": "Metric"})
    dataset_df = dataset_df.set_index("Metric")
    dataset_df = dataset_df.transpose()
    dataset_df = dataset_df.rename(columns={"chronos_bolt_base": "Chr.B.B",
                                            "chronos_bolt_small": "Chr.B.S",
                                            "chronos_bolt_mini": "Chr.B.M",
                                            "chronos-bolt-mini": "Chr.B.M",
                                            "chronos_bolt_tiny": "Chr.B.T",
                                            "timesfm1": "T.FM1",
                                            "timesfm2": "T.FM2"})
    dataset_df["Best"] = dataset_df.idxmin(axis=1) # add columns best
    return dataset_df

create_dataset_df(all_df, "bizitobs_application/10S/short")

Metric,Chr.B.B,Chr.B.S,Chr.B.M,Chr.B.T,T.FM1,T.FM2,Best
MASE,5.533290,5.406632,7.201342,7.547395,1391.842484,1.560993,T.FM2
Qloss,0.053958,0.034530,0.072335,0.069873,28.269510,0.014470,T.FM2
Rank,4.500000,3.000000,6.500000,7.000000,9.000000,1.500000,T.FM2


In [ ]:
final_df = pd.DataFrame()

for name in dataset_names:
    dataset_df = create_dataset_df(all_df, name)
    dataset_df["dataset, frequency, term"] = name.replace("/", ", ")

    dataset_df = dataset_df.reset_index()
    dataset_df = dataset_df[["dataset, frequency, term", "index", "Chr.B.B", "Chr.B.S", "Chr.B.M",
        "Chr.B.T", "T.FM1", "T.FM2", "Best"]]
    dataset_df = dataset_df.rename(columns={"index": "Metric"})
    dataset_df = dataset_df.rename_axis(index=" ")

    final_df = pd.concat([final_df, dataset_df], ignore_index=True)

In [49]:
final_df

Metric,"dataset, frequency, term",Metric,Chr.B.B,Chr.B.S,Chr.B.M,Chr.B.T,T.FM1,T.FM2,Best
0,"bizitobs_application, 10S, short",MASE,5.533290,5.406632,7.201342,7.547395,1391.842484,1.560993,T.FM2
1,"bizitobs_application, 10S, short",Qloss,0.053958,0.034530,0.072335,0.069873,28.269510,0.014470,T.FM2
2,"bizitobs_application, 10S, short",Rank,4.500000,3.000000,6.500000,7.000000,9.000000,1.500000,T.FM2
3,"bizitobs_application, 10S, medium",MASE,9.720338,9.146529,10.972406,11.416669,1199.144679,3.070241,T.FM2
4,"bizitobs_application, 10S, medium",Qloss,0.103502,0.084756,0.128187,0.115304,30.441737,0.032932,T.FM2
...,...,...,...,...,...,...,...,...,...
100,"solar, 10T, medium",Qloss,0.436094,0.452678,0.410149,0.494811,1120.471306,0.515808,Chr.B.M
101,"solar, 10T, medium",Rank,3.000000,4.500000,1.500000,6.000000,9.000000,7.500000,Chr.B.M
102,"solar, 10T, long",MASE,1.070707,1.188044,1.215812,1.288399,5320.462733,1.148564,Chr.B.B
103,"solar, 10T, long",Qloss,0.443021,0.496991,0.496322,0.534036,2088.004010,0.498298,Chr.B.B


In [50]:
final_df.to_csv("results/all_results.csv", index=False)

In [51]:
print(final_df.to_latex(index=False, float_format="%.3f"))

\begin{tabular}{llrrrrrrl}
\toprule
dataset, frequency, term & Metric & Chr.B.B & Chr.B.S & Chr.B.M & Chr.B.T & T.FM1 & T.FM2 & Best \\
\midrule
bizitobs_application, 10S, short & MASE & 5.533 & 5.407 & 7.201 & 7.547 & 1391.842 & 1.561 & T.FM2 \\
bizitobs_application, 10S, short & Qloss & 0.054 & 0.035 & 0.072 & 0.070 & 28.270 & 0.014 & T.FM2 \\
bizitobs_application, 10S, short & Rank & 4.500 & 3.000 & 6.500 & 7.000 & 9.000 & 1.500 & T.FM2 \\
bizitobs_application, 10S, medium & MASE & 9.720 & 9.147 & 10.972 & 11.417 & 1199.145 & 3.070 & T.FM2 \\
bizitobs_application, 10S, medium & Qloss & 0.104 & 0.085 & 0.128 & 0.115 & 30.442 & 0.033 & T.FM2 \\
bizitobs_application, 10S, medium & Rank & 4.500 & 3.000 & 6.500 & 7.000 & 9.000 & 1.500 & T.FM2 \\
bizitobs_application, 10S, long & MASE & 10.484 & 9.648 & 12.444 & 12.835 & 148.467 & 4.085 & T.FM2 \\
bizitobs_application, 10S, long & Qloss & 0.109 & 0.092 & 0.150 & 0.137 & 0.500 & 0.057 & T.FM2 \\
bizitobs_application, 10S, long & Rank & 4.5